# АНАЛИЗ ОБРАЗОВАТЕЛЬНЫХ МЕТРИК

Добро пожаловать в Edtech! Продакт-менеджер Василий попросил вас проанализировать поведение студентов на курсах, предметах и экзаменах онлайн-школы и ответить на следующие вопросы:

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)


2. Найдите и отсортируйте id экзаменов в рамках курса по возрастанию уровня завершаемости*.


3. Выявите самые популярные предметы (ТОП-3) по количеству регистраций на них.


4. Выявите предметы с самым большим оттоком (ТОП-3).


5. Используя pandas, в период с начала 2013 по конец 2014 выявите семестр с самой низкой завершаемостью курсов.


6. Используя pandas, в период с начала 2013 по конец 2014 выявите семестр с самыми долгими средними сроками сдачи курсов.


7. Часто для качественного анализа аудитории используют подходы, основанные на сегментации. Используя python, постройте адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можно выбрать следующие метрики:

    * R — среднее время сдачи одного экзамена,

    * F — завершаемость курсов,

    * M — среднее количество баллов, получаемое за экзамен.

Для каждого RFM-сегмента постройте границы метрик recency, frequency и monetary для интерпретации этих кластеров. Описание подхода можно найти тут.

     7.1. Чему равна минимальная граница по recency?
     7.2. Чему равна максимальная граница по recency?
     7.3. Чему равна минимальная граница по monetary?
     7.4. Чему равна максимальная граница по monetary?
     7.5. Сколько клиентов попадут в кластер 232?

Используйте логику:

    a) по recency можно получить 2, если значение recency меньше либо равно,  медиане по recency. В остальных случаях — 1.

    b) по frequency можно получить 1, если значение frequency меньше 50. Можно получить 2, если значение по frequency меньше 100. Можно получить 3 в остальных случаях.

    c) по monetary можно получить 1, если значение monetary меньше 40. Можно получить 2, если значение по monetary меньше либо равно 80. Можно получить 3 в остальных случаях.

Для решения задачи проведите предварительное исследование данных и сформулируйте, что должно считаться курсом. Обосновать свой выбор вы можете с помощью фактов сдачи экзаменов, распределения студентов и уникального идентификатора курса.

###### завершаемость = кол-во успешных экзаменов/кол-во всех попыток сдать экзамен * 100.

## Файлы:

### 1. Assessments.csv — файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).

- code_module — идентификационный код предмета.
- code_presentation — семестр (Идентификационный код).
- id_assessment — тест (Идентификационный номер ассессмента).
- assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).
- date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).
- weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%. Сумма всех остальных оценок составляет 100%.

### 2. Сourses.csv — файл содержит список предметов по семестрам.

- code_module — предмет (идентификационный код).
- code_presentation — семестр (идентификационный код).
- module_presentation_length — продолжительность семестра в днях.

### 3. StudentAssessment.csv — файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.

- id_assessment — тест (идентификационный номер).
- id_student — идентификационный номер студента.
- date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.
- is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).
- score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

### 4. StudentRegistration.csv — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.

- code_module — предмет (идентификационный код).
- code_presentation — семестр (идентификационный код).
- id_student — идентификационный номер студента.
- date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра. Например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала.
- date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

## Предварительный анализ данных

In [1]:
import pandas as pd
import numpy as np

### 1. Assessments.csv

In [29]:
assessments = pd.read_csv('assessments.csv')

In [30]:
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [31]:
# Размер файла
assessments.shape

(206, 6)

In [32]:
# Наличие пустых значений
assessments.isna().sum()

code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight                0
dtype: int64

In [33]:
# Типы данных
assessments.dtypes

code_module           object
code_presentation     object
id_assessment          int64
assessment_type       object
date                 float64
weight               float64
dtype: object

### 2. Courses.csv

In [13]:
courses = pd.read_csv('courses.csv')

In [14]:
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [15]:
# Размер файла
courses.shape

(22, 3)

In [16]:
# Наличие пустых значений
courses.isna().sum()

code_module                   0
code_presentation             0
module_presentation_length    0
dtype: int64

### 3. StudentAssessment.csv

In [17]:
studentAssessment = pd.read_csv('studentAssessment.csv')

In [18]:
studentAssessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [19]:
# Размер файла
studentAssessment.shape

(173912, 5)

In [20]:
# Наличие пустых значений
studentAssessment.isna().sum()

id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64

### 4. StudentRegistration.csv

In [22]:
studentRegistration = pd.read_csv('studentRegistration.csv')

In [23]:
studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [34]:
# Размер файла
studentRegistration.shape

(32593, 5)

In [35]:
# Наличие пустых значений
studentRegistration.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

### Кейс 1
Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)

In [36]:
# 1. Из studentAssessment выделить только те, которые сданы успешно, т.е. набрали 40+ баллов
# 2. Приджойнить к studentAssessment типы ассессментов из файла assessments
# 3. Отфильтровать только те ассесменты, которые относятся к типу Exam 
# 4. Посчитать количество уникальных успешно сданных экзаменов для каждого студента с помощью groupby
# 5. Посчитать количество студентов, у которых успешно сдан только 1 экзамен

task_1 = (
    studentAssessment.query('score > 39')
    .merge(
        assessments.query('assessment_type == "Exam"')[['id_assessment']].drop_duplicates(),
        on='id_assessment',
        how='inner'
    )
    .groupby('id_student', as_index=False)['id_assessment']
    .nunique()
    .rename(columns={'id_assessment': 'successes_count'})
)

# Подсчёт студентов с единственным успешным экзаменом
task_1 = task_1[task_1['successes_count'] == 1]['id_student'].nunique()

print(task_1)

3802


Мы выяснили, что 3802 студента успешно сдали только один курс.

### Кейс 2
Найдите и отсортируйте id экзаменов в рамках курса по возрастанию уровня завершаемости:

In [37]:
# 1. Из assessment выделить только тех, которые являются экзаменами
# 2. Приджойнить к studentAssessment типы ассессментов из файла assessments
# 3. Посчитать количество уникальных успешных попыток для каждого экзамена с помощью groupby
# 4. Посчитать общее количество попыток
# 5. Ввести новое поле - завершаемость


task_2 = (
    studentAssessment
        .merge(
            assessments.query('assessment_type == "Exam"')[['id_assessment']],
            on='id_assessment',
            how='inner'
        )
        .assign(success=lambda x: (x.score > 39).astype(int))
        .groupby('id_assessment', as_index=False)
        .agg(success_count=('success', 'sum'), total_attempts=('success', 'count'))
        .assign(completion_rate=lambda x: x.success_count / x.total_attempts)
        .sort_values(by='completion_rate')
)

task_2

,id_assessment,success_count,total_attempts,completion_rate
2,25340,504,602,0.837209
1,24299,1019,1168,0.872432
5,25368,842,950,0.886316
0,24290,664,747,0.888889
3,25354,878,968,0.907025
4,25361,485,524,0.925573


### Кейс 3
Выявите самые популярные предметы (ТОП-3) по количеству регистраций на них:

In [38]:
task_3 = (
    studentRegistration
        .groupby('code_module', as_index=False)
        .agg(registrations_count=('id_student', 'nunique'))  # Уникальное количество зарегистрировавшихся
        .sort_values(by='registrations_count', ascending=False)
        .head(3) 
)

task_3

,code_module,registrations_count
1,BBB,7692
5,FFF,7397
3,DDD,5848


### Кейс 4
Выявите предметы с самым большим оттоком* (ТОП-3):

*Отток в данном случае — это отмена регистрации на предмет.  

In [39]:
task_4 = (
    studentRegistration
        .query('date_unregistration.notna()')  
        .groupby('code_module', as_index=False)
        .agg(unregistrations_count=('id_student', 'nunique'))  # Уникальное количество зарегистрировавшихся
        .sort_values(by='unregistrations_count', ascending=False)
        .head(3) 
)

task_4

,code_module,unregistrations_count
1,BBB,2314
5,FFF,2249
3,DDD,2065


### Кейс 5
Используя pandas, в период с начала 2013 по конец 2014 выявите семестр с самой низкой завершаемостью курсов:

In [40]:
# 1. Из таблицы studentAssessment выделить только те экзамены, которые не были зачтены ранее
# 2. Рассчитать на основе поля score факт успешности экзамена
# 3. Для каждого семестра рассчитать среднюю завершаемость и средние сроки сдачи


task_5 = (
    studentAssessment
        .query('is_banked == 0')
        .merge(
            assessments.query('assessment_type == "Exam"')[['id_assessment', 'code_presentation']],
            on='id_assessment',
            how='inner'
        )
        .assign(
            success=lambda x: (x.score > 39).astype(int),
        )
        .groupby('code_presentation', as_index=False)
        .aggregate(
            {
                'success': [np.mean]
            }
        )
)

task_5

,code_presentation,success
,,mean
0,2013B,0.837209
1,2013J,0.907025
2,2014B,0.904013
3,2014J,0.878659


2013B - семестр с самой низкой завершаемостью курсов в период с начала 2013 по конец 2014.

### Кейс 6
Используя pandas, в период с начала 2013 по конец 2014 выявите семестр с самыми долгими средними сроками сдачи курсов:

In [42]:
# 1. Из таблицы studentAssessment выделить только те экзамены, которые не были зачтены ранее
# 2. Рассчитать на основе поля score факт успешности экзамена
# 3. Для успешных экзаменов выделить средние сроки их сдачи
# 4. Для каждого семестра рассчитать среднюю завершаемость и средние сроки сдачи



task_6 = (
    studentAssessment
        .query('is_banked == 0')
        .merge(
            assessments.query('assessment_type == "Exam"')[['id_assessment', 'code_presentation']],
            on='id_assessment',
            how='inner'
        )
        .assign(
            success=lambda x: (x.score > 39).astype(int),
            success_date_submitted=lambda x: x.success * x.date_submitted
        )
        .groupby('code_presentation', as_index=False)
        .aggregate(
            {
                'success_date_submitted': [lambda x: np.average(x, weights=(x > 0))]
            }
        )
)

task_6

,code_presentation,success_date_submitted
,,<lambda>
0,2013B,230.164683
1,2013J,239.509112
2,2014B,232.997389
3,2014J,243.680279


2014J - семестр с самыми долгими средними сроками сдачи курсов в период с начала 2013 по конец 2014.

### Кейс 7

Часто для качественного анализа аудитории используют подходы, основанные на сегментации. Используя python, постройте адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можете выбрать следующие метрики:

R — среднее время сдачи одного экзамена,

F — завершаемость курсов,

M — среднее количество баллов, получаемое за экзамен.

Для каждого RFM-сегмента постройте границы метрик recency, frequency и monetary для интерпретации этих кластеров.

In [43]:
# Получение идентификаторов экзаменов
exams_id = assessments.query('assessment_type == "Exam"')['id_assessment']

# Фильтрация результатов экзаменов и агрегация
exams_results = studentAssessment.query('id_assessment in @exams_id')

# Расчёт RFM метрик
rfm_exams = (
    exams_results
    .groupby('id_student', as_index=False)
    .agg(
        recency=('date_submitted', 'mean'),
        all_scores=('score', 'count')
    )
)

# Успешные экзамены
exams_success = exams_results.query('score >= 40')

# Расчёт успешных оценок
success_scores_by_students = (
    exams_success
    .groupby('id_student', as_index=False)
    .agg(success_scores=('score', 'count'))
)

# Объединение данных о RFM с успешными оценками
rfm_exams = rfm_exams.merge(success_scores_by_students, how='left', on='id_student').fillna(0)

# Расчёт частоты успешных оценок
rfm_exams['frequency'] = (100 * rfm_exams.success_scores / rfm_exams.all_scores).round(2)

# Расчёт средней оценки по всем экзаменам
all_scores_by_student = (
    exams_results
    .groupby('id_student', as_index=False)
    .agg(monetary=('score', 'mean'))
)

# Объединение с RFM данными
rfm_exams = rfm_exams.merge(all_scores_by_student, how='left', on='id_student').fillna(0)

# Выбор только необходимых столбцов
rfm_exams = rfm_exams[['id_student', 'recency', 'frequency', 'monetary']]

# Описание полученного DataFrame
print(rfm_exams.describe())

         id_student      recency    frequency     monetary
count  4.633000e+03  4633.000000  4633.000000  4633.000000
mean   7.256904e+05   238.462227    88.128642    65.117958
std    5.753498e+05     5.653378    32.114175    20.470561
min    2.369800e+04   229.000000     0.000000     0.000000
25%    5.011580e+05   234.000000   100.000000    50.000000
50%    5.884820e+05   241.000000   100.000000    66.000000
75%    6.463510e+05   243.000000   100.000000    82.000000
max    2.698251e+06   285.000000   100.000000   100.000000


In [44]:
rfm_exams.head()

,id_student,recency,frequency,monetary
0,23698,243.0,100.0,80.0
1,24213,236.0,100.0,58.0
2,27116,243.0,100.0,96.0
3,28046,237.0,100.0,40.0
4,28787,243.0,100.0,44.0


### Кейс 8
Используя логику:

1) по recency можно получить 2, если значение recency меньше либо равно медиане по recency. В остальных случаях — 1.

2) по frequency можно получить 1, если значение frequency меньше 50. Можно получить 2, если значение по frequency меньше 100. Можно получить 3 в остальных случаях.

3) по monetary можно получить 1, если значение monetary меньше 40. Можно получить 2, если значение по monetary меньше либо равно 80. Можно получить 3 в остальных случаях.

Сколько клиентов попадут в кластер 232?

In [45]:
def r_score(x):
    if x <= rfm_exams.recency.median():
        return 2
    else:
        return 1

def f_score(x):
    if x < 50:
        return 1
    elif 50 <= x < 100:
        return 2
    else:
        return 3

def m_score(x):
    if x < 40:
        return 1
    elif 40 <= x <=80:
        return 2
    elif x > 80:
        return 3

rfm_exams['R'] = rfm_exams['recency'].apply(lambda x: r_score(x))
rfm_exams['F'] = rfm_exams['frequency'].apply(lambda x: f_score(x))
rfm_exams['M'] = rfm_exams['monetary'].apply(lambda x: m_score(x))


rfm_exams['rfm_score'] = rfm_exams['R'].map(str) + rfm_exams['F'].map(str) + rfm_exams['M'].map(str)

In [46]:
rfm_exams.rfm_score.value_counts()

232    1555
132    1325
133     611
233     578
211     269
111     267
222      18
221       6
122       3
121       1
Name: rfm_score, dtype: int64

1555 клиентов попадут в кластер 232.

In [47]:
# Финальный датафрейм
rfm_exams.head()

,id_student,recency,frequency,monetary,R,F,M,rfm_score
0,23698,243.0,100.0,80.0,1,3,2,132
1,24213,236.0,100.0,58.0,2,3,2,232
2,27116,243.0,100.0,96.0,1,3,3,133
3,28046,237.0,100.0,40.0,2,3,2,232
4,28787,243.0,100.0,44.0,1,3,2,132
